# Train RL Agent

Trains both the network and the agent using the actual Minecraft environment

In [ ]:
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, Input, Lambda, Conv2D
from keras.optimizers import SGD, Adam
from keras import initializers
from keras import regularizers
import keras.backend as K

from anyrl.rollouts import replay

import time
import numpy as np

import minerl
import gym

import tensorflow as tf
import pickle
import collections
from collections import deque

import random

import sys
sys.path.append("../functions")

from ActionCombos import load_combos, get_all_action_combos, int_action_to_dict, convert_match_actions, match_batch_actions
from Utils import load_data, save_data
from DQfDModel import build_model, train_network

In [ ]:
resources_folder = 'new-resources'
models_path = 'app/models'
environment = 'Treechop'

# Training

In [ ]:
replay_file_path = "../{0}/buffer_{1}_post_train.sav".format(resources_folder, environment)
model_path = '../{0}/MineRL{1}-v0_model.h5'.format(models_path, environment)
unique_angles_file = '../{0}/unique_angles_{1}.sav'.format(resources_folder, environment.lower())
combos_file = '../{0}/action_combos_{1}.sav'.format(resources_folder, environment.lower())

action_combos = load_combos(combos_file)
unique_angles = load_data(unique_angles_file)

minerl_environment = 'MineRL{0}-v0'.format(environment)
n_action = len(action_combos)

action_keys = ['attack',
 'back',
  'camera',
 'forward',            
 'jump',
 'left',
 'right',
 'sneak',
 'sprint']

In [ ]:
env = gym.make(minerl_environment)

In [ ]:
expert_buffer = replay.PrioritizedReplayBuffer(500000, alpha=0.4, beta=0.6, epsilon=0.001)
expert_buffer = pickle.load(open(replay_file_path, 'rb'))
replay_buffer = replay.PrioritizedReplayBuffer(500000, alpha=0.4, beta=0.6, epsilon=0.001)

In [ ]:
tgt_model = build_model(n_action)
tgt_model.load_weights(model_path)

In [ ]:
init_model = build_model(n_action)
init_model.load_weights(model_path)

In [ ]:
train_network(env, init_model, tgt_model,expert_buffer,replay_buffer, n_action, action_combos, unique_angles, action_keys, max_timesteps=1000000)#,min_buffer_size=50